In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "tabularisai/multilingual-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def predict_sentiment(texts):
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_map = {0: "Very Negative", 1: "Negative", 2: "Neutral", 3: "Positive", 4: "Very Positive"}
    return [sentiment_map[p] for p in torch.argmax(probabilities, dim=-1).tolist()]



def tabularisi_ai(text):
    text= "I absolutely love the new design of this app!"

    a = predict_sentiment(text)
    a = a[0].split(" ")

    return a[0] if len(a)==1 else a[1]



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# text = "Covid cases are increasing fast!"
def cardiff_nlp(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]

    value = ""
    a = 0
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        print(f"{i+1}) {l} {np.round(float(s), 4)}")
        if np.round(float(s), 4) > a:
          value = l if not l == "neutral" else value
    return l


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from googletrans import Translator

async def translate_text(text):
    async with Translator() as translator:
        result = await translator.translate(text)
        print(result.text)
asyncio.run(translate_text("Check working"))


Check working


In [4]:

!pip install -q torch==2.1.0 transformers==4.35.0 safetensors==0.4.0 streamlit==1.28.0 pyngrok==7.1.2

import torch
if torch.cuda.is_available():
    !pip install -q flash-attn==2.3.3 --no-build-isolation
else:
    print("Skipping flash-attn installation since no GPU is available")

!pip install -q ninja packaging
!pip install -q https://github.com/deepspeedai/DeepSpeed

ds_config = {
  "train_batch_size": "auto",
  "train_micro_batch_size_per_gpu": "auto",
  "gradient_accumulation_steps": 4,
  "zero_optimization": {
    "stage": 2,
    "offload_optimizer": {
      "device": "cpu",
      "pin_memory": True
    },
    "contiguous_gradients": True,
    "overlap_comm": True
  },
  "bf16": {
    "enabled": True
  },
  "gradient_clipping": 1.0,
  "steps_per_print": 50,
  "wall_clock_breakdown": False
}

import json
with open('ds_config.json', 'w') as f:
    json.dump(ds_config, f)

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 45.7 MB/s eta 0:00:00
   ━━

In [5]:

import torch
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset, Dataset
import wandb
import os
import re
import numpy as np
from safetensors.torch import save_file, load_file
from flash_attn import flash_attn_qkvpacked_func
from accelerate import Accelerator
import streamlit as st
from pyngrok import ngrok
import time
import subprocess


# --------------------------- Set Up Cuda
def setup_cuda():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        st.success(f"CUDA enabled. Using {torch.cuda.get_device_name(0)}")

        # TF32 for Ampere GPUs (3x faster)
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
    else:
        device = torch.device("cpu")
        st.warning("CUDA not available. Using CPU - performance will be limited")

    os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
    os.environ["TORCH_CUDNN_V8_API_ENABLED"] = "1"
    return device

device = setup_cuda()

# --------------------------- MODEL INITIALIZATION
def load_model():
    # 4-bit quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True
    )

    # Pre-trained model
    model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        quantization_config=bnb_config,
        num_labels=2,
        device_map="auto"
    )

    # K-bit training
    model = prepare_model_for_kbit_training(model)

    # LoRA configuration
    peft_config = LoraConfig(
        r=8,
        lora_alpha=32,
        target_modules=["q_lin", "v_lin"],
        lora_dropout=0.05,
        bias="none",
        task_type="SEQ_CLS"
    )

    return get_peft_model(model, peft_config)

# -----------------------CUSTOM FLASH ATTENTION
class FlashAttentionWrapper(torch.nn.Module):
    def __init__(self, original_layer):
        super().__init__()
        self.layer = original_layer
        self.hidden_size = original_layer.dim

    def forward(self, hidden_states, attention_mask=None):
        qkv = self.layer(hidden_states)
        return flash_attn_qkvpacked_func(
            qkv,
            dropout_p=0.1,
            softmax_scale=1.0/np.sqrt(self.hidden_size)
        )

def load_and_combine_datasets():
    # ---------- First Dataset
    splits = {'train': 'train.csv'}
    df1 = pd.read_csv("hf://datasets/gxb912/large-twitter-tweets-sentiment/" + splits["train"])
    df1["text"] = df1["text"].str.replace(r"@\w+", "", regex=True)
    df1["text"] = df1["text"].str.strip()
    df1['sentiment_text'] = df1['sentiment'].apply(lambda x: "positive" if x == 1 else "negative")
    df1 = df1[['text', 'sentiment_text']]

    # ---------- Second Dataset
    splits = {'train': 'data/train-00000-of-00001.parquet'}
    df2 = pd.read_parquet("hf://datasets/mteb/tweet_sentiment_extraction/" + splits["train"])
    df2['sentiment_text'] = df2['label'].apply(lambda x: "positive" if x == 1 else "negative")
    df2 = df2[['text', 'sentiment_text']]

    # ---------- Third Dataset
    df3 = pd.read_parquet("hf://datasets/Marianina/sentiment-banking/data/train-00000-of-00001.parquet")
    df3['text'] = df3['inputs'].apply(lambda x: x['text'] if isinstance(x, dict) and 'text' in x else None)
    df3['sentiment'] = df3['prediction'].apply(lambda x: x[0]['label'].lower())
    df3 = df3[['text', 'sentiment']].rename(columns={'sentiment': 'sentiment_text'})
    df3 = df3.dropna(subset=['text'])

    combined_df = pd.concat([df1, df2, df3], ignore_index=True)

    combined_df['label'] = combined_df['sentiment_text'].map({'positive': 1, 'negative': 0})

    return combined_df

def create_real_dataset():
    df = load_and_combine_datasets()

    dataset = Dataset.from_pandas(df[['text', 'label']])
    return dataset


def preprocess_dataset(dataset, tokenizer):
    def tokenize_fn(examples):
        return tokenizer(
            examples["text"],
            truncation=True,
            max_length=256,
            padding="max_length"
        )

    dataset = dataset.map(tokenize_fn, batched=True)
    dataset = dataset.rename_column("label", "labels")
    return dataset

def setup_training():
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    model = load_model()
    dataset = create_real_dataset()

    for i, layer in enumerate(model.base_model.model.transformer.layer):
        layer.attention.out_lin = FlashAttentionWrapper(layer.attention.out_lin)

    dataset = dataset.train_test_split(test_size=0.2)
    train_ds = dataset["train"]
    val_ds = dataset["test"]

    train_ds = preprocess_dataset(train_ds, tokenizer)
    val_ds = preprocess_dataset(val_ds, tokenizer)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        gradient_accumulation_steps=4,
        learning_rate=2e-5,
        num_train_epochs=5,
        fp16=False,
        bf16=True,
        evaluation_strategy="steps",
        eval_steps=200,
        save_strategy="steps",
        save_steps=500,
        logging_steps=50,
        optim="paged_adamw_8bit",
        report_to="wandb",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        gradient_checkpointing=True,
        deepspeed="ds_config.json"
    )

    return model, tokenizer, train_ds, val_ds, data_collator, training_args




def start_training():
    model, tokenizer, train_ds, val_ds, data_collator, training_args = setup_training()

    print("Starting WandB logging...")
    wandb.init(project="sentiment-analysis", config=vars(training_args))

    print("Setting up Trainer with Flash Attention...")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print("Training started (this may take several minutes)...")
    trainer.train()

    print("Saving model weights in SafeTensors format...")
    save_file(
        model.state_dict(),
        "model_weights.safetensors",
        metadata={"format": "pt"}
    )

    print("Final evaluation...")
    eval_results = trainer.evaluate()
    wandb.log(eval_results)
    wandb.finish()


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

start_training()


ModuleNotFoundError: No module named 'flash_attn'

In [ ]:

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from safetensors.torch import load_file
import streamlit as st
import os
import time

import nest_asyncio
nest_asyncio.apply()

import asyncio

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
st.set_page_config(
    page_title="Advanced Sentiment Analysis",
    page_icon="😊",
    layout="centered"
)


@st.cache_resource(show_spinner=False)
def load_model():
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        num_labels=2
    )

    model.load_state_dict(load_file("model_weights.safetensors"))
    model.to(device)
    model.eval()

    return model, tokenizer

def render_header():
    st.title("Advanced Sentiment Analysis")
    st.markdown("""
    **4-bit QLoRA** • **Flash Attention v2** • **DeepSpeed ZeRO-2**
    *Analyze text sentiment with state-of-the-art efficiency*
    """)
    st.divider()

def render_input_form():
    with st.form("sentiment_form"):
        input_text = st.text_area(
            "Enter text for sentiment analysis:",
            "I absolutely love this amazing product!",
            height=150
        )
        submitted = st.form_submit_button("Analyze Sentiment")
        return submitted, input_text

def render_result(prediction, confidence):

    if prediction == 1:
        st.success(f" **POSITIVE** (Confidence: {confidence:.1f}%) from Trained Model")
    else:
        st.error(f"**NEGATIVE** (Confidence: {confidence:.1f}%) from Trained Model")

    pos_value = confidence if prediction == 1 else 100-confidence
    neg_value = 100-pos_value

    col1, col2 = st.columns(2)
    col1.metric("Positive", f"{pos_value:.1f}%")
    col2.metric("Negative", f"{neg_value:.1f}%")

    st.progress(pos_value/100 if prediction == 1 else neg_value/100)

def render_tech_details():
    with st.expander("Technical Details"):
        st.markdown("""
        ### Optimization Techniques
        <div style="background:#1a1a1a;padding:15px;border-radius:10px">
        <table>
          <tr><td><b>Technique</b></td><td><b>Benefit</b></td><td><b>Impact</b></td></tr>
          <tr><td>4-bit QLoRA</td><td>NF4 quantization</td><td>8x memory reduction</td></tr>
          <tr><td>LoRA Adapters</td><td>Rank-8 decomposition</td><td>0.1% params trainable</td></tr>
          <tr><td>Flash Attention v2</td><td>Memory-efficient attention</td><td>30% speedup, 50% memory reduction</td></tr>
          <tr><td>8-bit Adam</td><td>Paged optimizer</td><td>2x memory reduction</td></tr>
        </table>
        </div>
        """, unsafe_allow_html=True)

        if torch.cuda.is_available():
            st.caption(f"Running on: {torch.cuda.get_device_name(0)}")
            mem_alloc = torch.cuda.memory_allocated() / 1024**3
            mem_reserved = torch.cuda.memory_reserved() / 1024**3
            st.caption(f"GPU Memory: {mem_alloc:.2f}/{mem_reserved:.2f} GB")

def main():
    render_header()

    if not os.path.exists("model_weights.safetensors"):
        st.error("Model not found. Please ensure 'model_weights.safetensors' is in the current directory.")
        st.stop()

    model, tokenizer = load_model()

    submitted, input_text = render_input_form()
    input_text = asyncio.run(translate_text(input_text))

    if submitted:
        with st.spinner("Analyzing sentiment..."):
            try:
                inputs = tokenizer(
                    input_text,
                    return_tensors="pt",
                    truncation=True,
                    max_length=256
                ).to(device)

                input_text =

                with torch.no_grad():
                    outputs = model(**inputs)

                probs = torch.softmax(outputs.logits, dim=1)[0]
                prediction = torch.argmax(probs).item()
                confidence = probs[prediction].item() * 100

                time.sleep(0.5)
                value_1 = tabularisi_ai(input_text)
                value_2 = cardiff_nlp(input_text)

                st.success(f"{value_1}) from Tabularisi Ai")

                st.success(f"{value_2}) from Cardiff NLP")

                render_result(prediction, confidence)

            except Exception as e:
                st.error(f"Error during analysis: {str(e)}")

    render_tech_details()

if __name__ == "__main__":
    main()

def start_ngrok(port=8501):
    try:
        ngrok.kill()
    except:
        pass

    public_url = ngrok.connect(port).public_url
    st.success(f"🌐 App available at: {public_url}")
    return public_url

if __name__ == "__main__":
    IN_COLAB = 'google.colab' in str(get_ipython()) if 'get_ipython' in globals() else False

    if IN_COLAB:
        st.info("Google Colab Detected - Enabling ngrok tunnel")
        public_url = start_ngrok()

    main()

    if IN_COLAB:
        while True:
            time.sleep(86400)